<h1 style = "font-family: garamond; font-size:50px; font-style: normal;background-color: #f6f5f5; color :#c97820; border-radius: 10px 20px; text-align:center">PetFinder.my - Pawpularity with <span style="color: #8532a8"> Pytorch ⚡ </span> </h1>

![](https://storage.googleapis.com/kaggle-competitions/kaggle/25383/logos/header.png)

<p style="color :#006699; font-size:20px" > Millions of stray animals suffer on the streets or are euthanized in shelters every day around the world. You might expect pets with attractive photos to generate more interest and be adopted faster. But what makes a good picture? With the help of data science, you may be able to accurately determine a pet photo’s appeal and even suggest improvements to give these rescue animals a higher chance of loving homes.
</p>

<p style="font-size:20px; background-color: #f6f5f5; color :#6666ff; border-radius: 10px 10px; text-align:center" > 
    What will we learn today?
</p>

<p style="font-size:17px; background-color: #f6f5f5; color :#a85e32; border-radius: 10px 10px; text-align:center">
    Developing and Training models with Pytorch Lightning ⚡ <br>
    Stratified Kfold with Pytorch Lightning ⚡
    </p>
    
    
 <p style="font-size:30px; background-color: #f6f5f5; color :#e6cf25; border-radius: 10px 10px; text-align:center" > 
   🌩️ Sit back and enjoy the Lightning show! 🌩️
</p>
    


<h1 style = "font-family: garamond; font-size:45px; font-style: normal; color :#c97821; border-radius: 10px 20px; text-align:left">Library Installs</h1>

In [ ]:
!pip install -q timm
!pip install -q pytorch-lightning

<h1 style = "font-family: garamond; font-size:45px; font-style: normal; color :#c97821; border-radius: 10px 20px; text-align:left">Imports</h1>

In [ ]:
# Pytorch Lightning Utils
import pytorch_lightning as pl
from pytorch_lightning import Callback
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import WandbLogger

# Pytorch 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim.optimizer import Optimizer, required 
import timm


# ML Utils
from sklearn import model_selection
from sklearn.metrics import mean_squared_error
import cv2
import numpy as np
import pandas as pd
from PIL import Image

# Visualization Utils
from matplotlib import pyplot as plt
import plotly.graph_objects as go
import plotly_express as px
import plotly.io as pio
pio.templates.default = "plotly_dark"

import seaborn as sns

from colorama import Fore, Back, Style
r_ = Fore.RED
g_ = Fore.GREEN
b_ = Fore.BLUE
c_ = Fore.CYAN
y_ = Fore.YELLOW
res = Style.RESET_ALL

# Image Augmentation Library
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Misc Utils
import os
import random

# Hide Warning
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

<h1 style = "font-family: garamond; font-size:45px; font-style: normal; color :#c97821; border-radius: 10px 20px; text-align:left">Reading the Data</h1>

<p style="font-family: garamound; font-size:20px;" > Lets read the data and add full file path for easy reading </p>

In [ ]:
# reading the files
train = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
test = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')

# functions to append full path
def get_train_file_path(image_id):
    return "../input/petfinder-pawpularity-score/train/{}.jpg".format(image_id)

def get_test_file_path(image_id):
    return "../input/petfinder-pawpularity-score/test/{}.jpg".format(image_id)

train['file_path'] = train['Id'].apply(get_train_file_path)
test['file_path'] = test['Id'].apply(get_test_file_path)

<p style="font-family: garamound; font-size:20px;" > Lets see what we are working with </p>

In [ ]:
print(f"{r_}TRAINING DATA{res}")
train.head().style.set_properties(**{'background-color': 'lightyellow',
                           'color': 'brown'})

In [ ]:
print(f"{g_}TESTING DATA{res}")
test.head().style.set_properties(**{'background-color': 'lightyellow',
                           'color': 'brown'})

<h1 style = "font-family: garamond; font-size:45px; font-style: normal; color :#c97821; border-radius: 10px 20px; text-align:left">Visualizing the <span style="color: #bf4f26" > Target Variable </span> </h1>

<p style="font-family: garamound; font-size:20px;" > A Histogram plot of the pawpularity score </p>

In [ ]:
# create the bins
counts, bins = np.histogram(train['Pawpularity'], bins=range(0, 110, 10))
bins = 0.5 * (bins[:-1] + bins[1:])

fig = px.bar(x=bins, y=counts)
             
fig.update_layout(title_text='Pawpularity distribution', 
                  xaxis=dict(title='Pawpularity'),
                  yaxis=dict(title='Count'),
                  showlegend=False
                 )
fig.show()

<h1 style = "font-family: garamond; font-size:45px; font-style: normal; color :#c97821; border-radius: 10px 20px; text-align:left">Configuring the Hyperparameters </h1>

<p style="font-family: garamound; font-size:20px;" > You can change hyperparameters here to see how it affects the final model </p>

In [ ]:
Config = {
    'seed' : 2021,  # setting the seed
    'debug': False,  # debug mode
    'num_folds': 5, # number of folds to split
    'trn_folds': [0, ],  # folds to train
    'batch_size': 12,  # batch size
    'num_workers' : 8,  # number of cpu workers
    'img_size': 512,  # image size
    'model': 'efficientnet_b3',  # model to use
    'img_ftr_len': 500, # length of vector to encode the images in
    'epochs': 15,        # numbers of epochs
    'precision': 16,
    'weight_decay': 1e-6,
    
    # learning
    'lr': 1e-4,
    'T_max': 10,
    'min_lr': 1e-6,
    'patience_earlystop': 3,
}   

In [ ]:
if Config['debug']:
    train = train[:500].reset_index(drop=True)

<h1 style = "font-family: garamond; font-size:45px; font-style: normal; color :#c97821; border-radius: 10px 20px; text-align:left">Seeding for Reproducibility</h1>

<p style="font-family: garamound; font-size:20px;" > We seed everything, so that we can reproduce the results everytime </p>

In [ ]:
def seed_everything(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(Config['seed'])

<h1 style = "font-family: garamond; font-size:45px; font-style: normal; color :#c97821; border-radius: 10px 20px; text-align:left"> Creating Folds </h1>

<p style="font-family: garamound; font-size:20px;" > We create Stratified Kfolds of the data. We first bin values into discrete intervals with <span style="font-weight: bold"> pd.cut </span> and then split</p>

In [ ]:
# https://www.kaggle.com/abhishek/same-old-creating-folds
def create_folds(data, num_splits, seed):
    data["kfold"] = -1
    num_bins = int(np.floor(1 + np.log2(len(data))))

    data.loc[:, "bins"] = pd.cut(data["Pawpularity"], bins=num_bins, labels=False)

    kf = model_selection.StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=seed)
    
    for f, (t_, v_) in enumerate(kf.split(X=data, y=data.bins.values)):
        data.loc[v_, 'kfold'] = f
    
    data = data.drop("bins", axis=1)

    return data

train = create_folds(train, Config['num_folds'], Config['seed'])

<h1 style = "font-family: garamond; font-size:45px; font-style: normal; color :#c97821; border-radius: 10px 20px; text-align:left"> 
Dataset Class</h1>

In [ ]:
class PetPawpularityDs(Dataset):
    def __init__(self, df, split='train', transform=None):
        self.df = df
        self.transforms = transform
        self.ftrcols = [col for col in df.columns if col not in ['Id', 'Pawpularity', 'file_path', 'kfold']]
        self.split = split
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_path = self.df.loc[idx, 'file_path']
        img = cv2.imread(img_path)
        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        ftrs = torch.Tensor(np.array(self.df.loc[idx, self.ftrcols].values).astype(np.float32))
        
        if self.split == 'train':
            score = torch.Tensor([self.df.loc[idx, 'Pawpularity']])
        
        if self.transforms:
            img = self.transforms(image=img)["image"]
        
        if self.split=='train':
            return img, ftrs, score
        else:
            return img, ftrs

<h1 style = "font-family: garamond; font-size:45px; font-style: normal; color :#c97821; border-radius: 10px 20px; text-align:left"> 
Augmentations </h1>

<p style="font-family: garamound; font-size:20px;" > We will just use a Horizontal Flip augmentation for Baseline. Feel free to test out more augmentations here</p>

In [ ]:
def get_train_transforms():
    return A.Compose([
                        A.Resize(Config['img_size'], Config['img_size']),
                        A.HorizontalFlip(p=0.5),
                        A.Normalize(
                                mean=[0.485, 0.456, 0.406], 
                                std=[0.229, 0.224, 0.225], 
                            ),
                        ToTensorV2()
                    ])


def get_valid_transforms():
    return A.Compose([
                        A.Resize(Config['img_size'], Config['img_size']),
                        A.Normalize(
                            mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225],
                        ),
                        ToTensorV2(),
                ])

def get_test_transforms():
    return A.Compose([
                    A.Resize(Config['img_size'], Config['img_size']),
                    A.Normalize(
                            mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225],
                        ),
                    ToTensorV2(),
                ])

<p p style = "font-family: garamond; font-size:30px; font-style: normal;background-color: #f6f5f5; color :#6666ff; border-radius: 10px 10px; text-align:center">Pytorch Lightning DataModule ⚡</p>

<p style="font-family: garamound; font-size:20px;" > A datamodule is a shareable, reusable class that encapsulates all the steps needed to process data. Simply put, a DataModule is simply a collection of a train_dataloader, val_dataloader(s), test_dataloader(s) along with the matching transforms and data processing/downloads steps required.</p>

In [ ]:
# custom collate functions to combine image, features and labels

def train_collate_fn(data):
    
    images = torch.zeros((len(data), 3, Config['img_size'], Config['img_size']))
    datafeatures = torch.zeros((len(data), 12))
    scores = torch.zeros((len(data), 1))
    
    for i in range(len(data)):
        images[i, ...] = data[i][0]
        datafeatures[i, ...] = data[i][1]
        scores[i, ...] = data[i][2]
        
    return images.float(), datafeatures.float(), scores.float()

def test_collate_fn(data):
    
    images = torch.zeros((len(data), 3, Config['img_size'], Config['img_size']))
    datafeatures = torch.zeros((len(data), 12))
    
    for i in range(len(data)):
        images[i, ...] = data[i][0]
        datafeatures[i, ...] = data[i][1]
        
    return images.float(), datafeatures.float()


# Pytorch lightning Data Module
class PawpularityDModule(pl.LightningDataModule):

    def __init__(self, df, fld):
        super().__init__()
        self.fold = fld
        self.train_data = df[df['kfold'] != self.fold].reset_index(drop=True)
        self.val_data = df[df['kfold'] == self.fold].reset_index(drop=True)
        self.test_data = test
        
    def setup(self, stage=None):
        self.train_dataset = PetPawpularityDs(
                df = self.train_data,
                transform = get_train_transforms()    
            )
        
        self.val_dataset = PetPawpularityDs(
                df = self.val_data,
                transform = get_valid_transforms(),
        )

        self.test_dataset = PetPawpularityDs(
                df = self.test_data,
                transform = get_test_transforms(),
                split='test'
        )

    def train_dataloader(self):
        return DataLoader(
                self.train_dataset,
                batch_size=Config['batch_size'],
                shuffle=True,
                num_workers=Config['num_workers'],
                pin_memory=True, 
                collate_fn = train_collate_fn
            )

    def val_dataloader(self):
        return DataLoader(
                self.val_dataset,
                batch_size = Config['batch_size'],
                shuffle=False,
                num_workers = Config['num_workers'],
                pin_memory=True,
                collate_fn = train_collate_fn
            )

    def test_dataloader(self):
        return DataLoader(
                self.test_dataset, 
                batch_size = Config['batch_size'],
                shuffle=False, 
                num_workers = Config['num_workers'],
                pin_memory=True,
                collate_fn = test_collate_fn
            )

<h1 style = "font-family: garamond; font-size:45px; font-style: normal; color :#c97821; border-radius: 10px 20px; text-align:left"> 
Training Data Visualization </h1>

In [ ]:
# create a data module and set up
dm = PawpularityDModule(train, 0)
dm.setup()

# Plot some training images
import torchvision.utils as vutils
img, ftrs, targets = next(iter(dm.train_dataloader()))

plt.figure(figsize=(16, 8))
plt.axis("off")
plt.title("Training Images", fontsize=30)
_ = plt.imshow(vutils.make_grid(
    img[:12], nrow=4, padding=7, normalize=True).cpu().numpy().transpose((1, 2, 0)))

print(f"{g_} {targets[:12].reshape((3,4))} {res}\n\n")

<p p style = "font-family: garamond; font-size:30px; font-style: normal;background-color: #f6f5f5; color :#6666ff; border-radius: 10px 10px; text-align:center">Pytorch Lightning Module ⚡</p>

<p style="font-family: garamound; font-size:20px;" > A LightningModule organizes your PyTorch code into 5 sections<br>1. Computations (Models).<br>2. Train loop (training_step)<br>3. Validation loop (validation_step)<br>4. Test loop (test_step)<br>5. Optimizers (configure_optimizers) <br> <br>  We define the Model here in the <span style="font-weight: bold"> Trainer </span> class. Here's a snapshot of the model we are going to build, </p>


In [ ]:
from IPython.display import Image
Image(filename='../input/imagestore/finalmodel.png') 


In [ ]:
class Trainer(pl.LightningModule):

    def __init__(self, model_name = Config['model'],out_features=Config['img_ftr_len'],
                 pretrained=True):
        super().__init__()
        
        # feature extractor
        self.model = timm.create_model(model_name, pretrained=pretrained,
                                       num_classes = out_features)
        
        # final fc layer
        self.finalfc = nn.Sequential(
            nn.Linear(out_features+12, 120),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(120, 1)
        )
        
        self.criterion = nn.MSELoss()
                
    def forward(self, img, ftrs):
        
        # feature extractor extracts features from the image
        imgouts = self.model(img)
 
        # we combine the meta features with the image features given
        ftrout = torch.cat([imgouts, ftrs], dim=-1)
        
        # we then pass the combined feature into final layer
        output = self.finalfc(ftrout)
    
        return output


    def training_step(self, batch, batch_idx):

        img, ftrs, score = batch
        output = self.forward(img, ftrs)
        
        loss = self.criterion(output, score)
        
        try:
            rmse = mean_squared_error(score.detach().cpu(), output.detach().cpu(), squared=False) 

            self.log("RMSE", rmse, on_step= True, prog_bar=True, logger=True)
            self.log("Train Loss", loss, on_step= True,prog_bar=False, logger=True)
        
        except:
            pass

        return {"loss": loss, "predictions": output.detach(), "labels": score.detach()}

    def training_epoch_end(self, outputs):

        preds = []
        labels = []
        
        for output in outputs:
            
            preds += output['predictions']
            labels += output['labels']

        labels = torch.stack(labels)
        preds = torch.stack(preds)

        train_rmse = mean_squared_error(labels.detach().cpu(), preds.detach().cpu(), squared=False)
        
        self.print(f'Epoch {self.current_epoch}: Training RMSE: {train_rmse:.4f}')
        
        self.log("mean_train_rmse", train_rmse, prog_bar=False, logger=True)

    def validation_step(self, batch, batch_idx):
        
        img, ftrs, score = batch
        
        with torch.no_grad():
            output = self.forward(img, ftrs)

            loss = self.criterion(output, score)
        
        self.log('val_loss', loss, on_step= True, prog_bar=False, logger=True)
        return {"predictions": output.detach(), "labels": score}
      

    def validation_epoch_end(self, outputs):

        preds = []
        labels = []
        
        for output in outputs:
            preds += output['predictions']
            labels += output['labels']

        labels = torch.stack(labels)
        preds = torch.stack(preds)

        val_rmse = mean_squared_error(labels.detach().cpu(), preds.detach().cpu(), squared=False)
        
        self.print(f'Epoch {self.current_epoch}: Validation RMSE: {val_rmse:.4f}')

        
        self.log("val_rmse", val_rmse, prog_bar=True, logger=True)
        

    def test_step(self, batch, batch_idx):
        img, ftrs = batch
        output = self.forward(img, ftrs)
        
        return output   

    def configure_optimizers(self):

        param_optimizer = list(self.model.named_parameters())
        
        # configuring parameters
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        optimizer_parameters = [
            {
                "params": [
                    p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
                ],
                "weight_decay": Config['weight_decay'],
            },
            {
                "params": [
                    p for n, p in param_optimizer if any(nd in n for nd in no_decay)
                ],
                "weight_decay": 0.0,
            },
        ]
    
        # we use adam optimizer with Cosine annealing LR
        optimizer = Adam(optimizer_parameters, lr=Config['lr'])
        
        
        scheduler = CosineAnnealingLR(optimizer,
                              T_max=Config['T_max'],
                              eta_min=Config['min_lr'],
                              last_epoch=-1)

        return dict(
          optimizer=optimizer,
          lr_scheduler=scheduler
        )

<p p style = "font-family: garamond; font-size:30px; font-style: normal;background-color: #f6f5f5; color :#6666ff; border-radius: 10px 10px; text-align:center">Pytorch Lightning Trainer ⚡</p>

<p style="font-family: garamound; font-size:20px;" > Once you’ve organized your PyTorch code into a LightningModule, the Trainer automates everything else.<br><br> This ensures following:<br><br>1. You maintain control over all aspects via PyTorch code without an added abstraction.<br>2. The trainer uses best practices embedded by contributors and users from top AI labs such as Facebook AI Research, NYU, MIT, Stanford, etc…<br>3. The trainer allows overriding any key part that you don’t want automated. <br> <br> While keeping track of RMSE we save the model checkpoints </p>

In [ ]:
model = Trainer()
Training_metrics = []

for fold in Config['trn_folds']:
      
    print(f"{'='*38} Fold: {fold} {'='*38}")
    
    # create data module
    data_module = PawpularityDModule(train, fold)
    
    # stop the training early
    early_stopping_callback = EarlyStopping(monitor='val_rmse',mode="min", patience=Config['patience_earlystop'])
    
    # store model checkpoints
    checkpoint_callback = ModelCheckpoint(
      dirpath="checkpoints",
      filename="best-checkpoint-{fold}-{val_rmse:.3f}",
      save_top_k = Config['epochs'],
      verbose=True,
      monitor="val_rmse",
      mode="min"
    )
    
    # define trainer
    trainer = pl.Trainer(
      gpus = 1,
      checkpoint_callback=True,
      callbacks=[early_stopping_callback,checkpoint_callback],
      max_epochs = Config['epochs'],
      precision = Config['precision'],
      progress_bar_refresh_rate=1, 
      num_sanity_val_steps=1 if Config['debug'] else 0,
      stochastic_weight_avg = True,
    )
    
    # fit
    trainer.fit(model, data_module) 

In [ ]:
!ls checkpoints

<p p style = "font-family: garamond; font-size:30px; font-style: normal; color :#fcba03; border-radius: 10px 10px; text-align:center">Acknowledgements</p>

<p style="font-family: garamound; font-size:20px;" > Some notebooks which were extemely helpful in the making of this - <br> <a href="https://www.kaggle.com/ligtfeather/pl-1fold-cqt-deepspeed-op-baseline-w-b-84"> https://www.kaggle.com/ligtfeather/pl-1fold-cqt-deepspeed-op-baseline-w-b-84 </a> <br> <a href="https://www.kaggle.com/hmendonca/melanoma-neat-pytorch-lightning-native-amp"> https://www.kaggle.com/hmendonca/melanoma-neat-pytorch-lightning-native-amp </a> <br>
<a href="https://www.kaggle.com/abhishek/same-old-creating-folds"> https://www.kaggle.com/abhishek/same-old-creating-folds </a> </p>


